In [ ]:
from datetime import datetime

GET_FROM: datetime = datetime.fromisoformat("2025-01-01T00:00:00+03:00")
GET_TO: datetime = datetime.fromisoformat("2025-10-01T00:00:00+03:00")

ENTITIES_LOG_COUNT: int = 10
DOC_LIMIT: int = int(60 * 60 * 8 / 1.3)  # ~8 hours of scraping

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve() / "src"))

from savers.mongo import MongoSaver
from scrapers.rbc import RBCGetter, RBCGetterType, RBCParser
from log import log

import logging

log.setLevel(logging.INFO)


def log_first_n(set: set, n: int):
    i = 0
    for s in set:
        if i >= n:
            break
        log.info(f"\t{s}")
        i += 1

In [ ]:
getter = RBCGetter(1.2, RBCGetterType.Sitemap, GET_FROM, GET_TO, DOC_LIMIT)
parser = RBCParser()

In [ ]:
sources = getter.fetch_sources()
log.info(f"first {ENTITIES_LOG_COUNT} sources = ")
log_first_n(sources, ENTITIES_LOG_COUNT)

In [ ]:
scrap = getter.fetch_scrap(sources)

In [ ]:
parsed_scrap = parser.parse_scrap(scrap)

In [ ]:
parser.info_scrap(scrap)
parser.info_parsed_scrap(parsed_scrap)

In [ ]:
log.info(f"first {ENTITIES_LOG_COUNT} parsed scraps = ")
log_first_n(set(map(lambda x: x.value, parsed_scrap)), ENTITIES_LOG_COUNT)

In [ ]:
saver = MongoSaver()
saver.save_parsed_scrap(parsed_scrap)